In [1]:
#Libraries
import zipfile, os
import shutil
import tarfile
import re
import xml.etree.ElementTree as etree
import pandas as pd
from bs4 import BeautifulSoup
from xml.etree import ElementTree
import os
import json
import pandas as pd
import re
from nltk.tokenize import sent_tokenize
from pdfrw import PdfReader
import numpy as np
from nltk import ngrams
import scispacy
import spacy

In [147]:
#NLM Function
def get_from_NLM(RootDir1,TargetFolder,molecule_name,publisher_name):
    abstract_list=[]
    section_list=[]
    xml_title_list=[]
    actual_title_list=[]
    actual_journal_list=[]
    actual_doi_list=[]
    actual_url_list=[]
    copied_count = 0
    for root, dirs, files in os.walk((os.path.normpath(RootDir1)), topdown=False):         
        for name in files:
            SourceName = os.path.join(root,name)
            xml_name=name
            with open(SourceName,'rb') as name:
                data = name.read()
                text = str(data)
                shorttext = str(data)[:100]
                if 'NLM' in shorttext:
                    #shutil.copy2(SourceName, TargetFolder) #copying the files to target folder
                    copied_count = copied_count + 1

                    tree = etree.parse(SourceName)
                    roots = tree.getroot()

                    ###Abstract 
                    abs_dict = {}  
                    abstract_title = ""
                    for tag in roots.iter("abstract"):
                        tagtext = ElementTree.tostring(tag)
                        tagtext= (str(tagtext, 'UTF-8'))
                        cleanr = re.compile(r'(<(?!/abstract|abstract|p|/p|title|/title).*?>)|(\n)')
                        tagtext = bytes(cleanr.sub('', tagtext),'UTF-8')
                        
                        root_1 = ElementTree.fromstring(tagtext)

                        for elem in root_1.iter():
                            if elem.tag == "title":
                                abstract_title = elem.text
                            if elem.tag == "p":
                                if (elem.text)==None:
                                    pass
                                elif abstract_title in abs_dict:
                                    abs_dict[abstract_title] = abs_dict[abstract_title]+" "+str(elem.text)
                                else:
                                    abs_dict[abstract_title] = elem.text
                    abstract_list.append("".join(abs_dict.values()))
                    #print ("----------------------------------------------------------------------------------")

                    ###Body Section
                    section_dict = {}  
                    section_title = ""
                    for tag in roots.iter("body"):
                        tagtext = ElementTree.tostring(tag)  
                        tagtext= (str(tagtext, 'UTF-8'))
                        tagtext=re.sub("(<[table(.+?)]>(.+?)</[table]>)|(<[fig(.+?)]>(.+?)</[fig]>)", '', tagtext)
                        cleanr = re.compile(r'(<(?!/body|body|p|/p|table|/table|title|/title).*?>)|(\n)')
                        tagtext = bytes(cleanr.sub('', tagtext),'UTF-8')

                        root_1 = ElementTree.fromstring(tagtext) 

                        for elem in root_1.iter():
                            if elem.tag == "title":
                                section_title = elem.text
                            if elem.tag == "p":
                                if (elem.text)==None:
                                    pass
                                elif section_title in section_dict:
                                    section_dict[section_title] = section_dict[section_title]+" "+str(elem.text)
                                else:
                                    section_dict[section_title] = elem.text
                    section_dict={k: v for k, v in section_dict.items()}
                    section_list.append(str(section_dict))
                    #print ("----------------------------------------------------------------------------------")

                    ###Title, DOI, URL
                    for tag in roots.iter('front'):
                        tagtext = ElementTree.tostring(tag) 
                        tagtext= (str(tagtext, 'UTF-8'))
                        cleanr = re.compile(r'(<(?!/article-title|article-title|front|/front|journal-title|/journal-title|article-id|/article-id).*?>)|(\n)')
                        tagtext = bytes(cleanr.sub('', tagtext),'UTF-8')
                        root_1 = ElementTree.fromstring(tagtext)               
                        title_sen=""
                        #journal_sen=""
                        pmc_sen=""
                        doi_sen=""
                        url_sen=""
                        for elem in root_1.iter():
                            if elem.tag == "article-title":                    
                                title_sen = title_sen+elem.text
                            #if elem.tag =="journal-title":
                                #journal_sen= journal_sen+elem.text
                            if elem.attrib == {'pub-id-type':'pmc'}:
                                pmc_sen= pmc_sen+elem.text
                            if elem.attrib == {'pub-id-type':'doi'}:
                                doi_sen= doi_sen+elem.text 
                        if doi_sen=="":
                            if (pmc_sen.startswith('PMC')):
                                url_sen= 'https://www.ncbi.nlm.nih.gov/pmc/articles/%s/'%pmc_sen
                                doi_sen = pmc_sen
                            else:
                                url_sen= 'https://www.ncbi.nlm.nih.gov/pmc/articles/PMC%s/'%pmc_sen
                                doi_sen = "PMC"+pmc_sen
                        else:
                            url_sen= "http://dx.doi.org/"+doi_sen     

                        #actual_journal_list.append(journal_sen)
                        actual_title_list.append(title_sen)
                        actual_doi_list.append(doi_sen) ###DOI
                        actual_url_list.append(url_sen) ###URL
                        xml_title_list.append(xml_name)



        NLM_data = pd.DataFrame(list(zip(xml_title_list, actual_title_list,abstract_list,section_list,actual_doi_list,actual_url_list)),
                   columns =['XML','Title','Abstract_full','Full_body_text','DOI','URL']).replace('','None')

        NLM_data['Molecule'] = molecule_name
        NLM_data['Publisher_type'] = publisher_name

        return NLM_data               

In [148]:
#************
RootDir1 = r'Downloads/HAWK_AI_2022/Brodalumab/NLM' #Input files location
TargetFolder = r'Downloads/HAWK_AI_2022/Brodalumab/NLM' #output files location
#*****************
NLM_data = get_from_NLM(RootDir1,TargetFolder,'Brodalumab','NLM') #format-RootDir1,TargetFolder,molecule_name,publisher_name

In [149]:
NLM_data

XML  \
0    021e6342-faf8-4f25-a290-eed3d68427ef.xml   
1    03403df2-e5ab-4176-9d0f-28a9e0a39783.xml   
2    03fa7237-f7b4-4f47-a9e7-1bfc2b694c1c.xml   
3    06581fe8-9e74-48f4-aeea-d344f5b71c38.xml   
4    079bef58-9946-4ef5-a052-b12fcbb70061.xml   
5    082f257f-917b-45f4-bbec-af6eef656bd8.xml   
6    084a6d9e-1e49-4b85-9ac3-5cf0d195d941.xml   
7    08db5b78-3bb3-4dd6-88f2-f76d4c2a4037.xml   
8    08ed31ce-5eb9-41a6-96e6-e0984c3c63dc.xml   
9    09b38fba-5899-4b38-bf0a-e78ad044c98f.xml   
10   09c263f3-7b88-4b28-8bb7-aa414e0ab9a5.xml   
11   0a16e269-f7ab-4562-80fc-3c30951cb7fd.xml   
12   0a637208-546c-424d-abe3-c91a0786b943.xml   
13   0ae0f9a2-b60f-4c5d-a1ba-b44a6851c270.xml   
14   0b9289a3-ea2b-451a-9160-3b557fc670e2.xml   
15   0bb8f05b-2f13-417b-8b13-429caf213bf9.xml   
16   0bc312b9-a280-4d2c-8ebe-92f48585707f.xml   
17   0bcaa453-0bb5-4fde-9b0d-6dd69ec99331.xml   
18   0bd9fffc-19bd-45ac-aafb-e748e1b7ada0.xml   
19   0d3932bc-be4b-4160-b19a-f61de11a3f63.xml   
20   0e52da66-76d1-4121-921c-43fd3d88bf8e.xml   
21   0f1da0f5-457e-482e-a652-468841abb1d2.xml   
22   0f9085ee-490f-4215-a3af-3fc83bbd9843.xml   
23   0fc38272-a8d6-48aa-b874-af91b0e8efd6.xml   
24   0fed9a79-7728-4754-94dd-b73fb14dd40b.xml   
25   10841247-1022-428f-b2c8-392b1f6e8edc.xml   
26   10b5d18a-79ef-4230-a0e1-2600c3e1a018.xml   
27   136bdaf8-a027-4b58-8f6f-24ce0dee5340.xml   
28   144bc614-6333-4d42-b35d-26cb3958b1c4.xml   
29   15090da9-8281-4988-915e-a05d1d52e454.xml   
..                                        ...   
349  e9f6b0e5-69fc-4364-9336-e400a0d81695.xml   
350  e9f87265-36c5-4769-a806-906dd8df05f4.xml   
351  ebc0b55c-e648-4d93-b25b-92cd74cf8a41.xml   
352  ebec3728-138b-4dba-9803-52fa024571bd.xml   
353  ec32935a-b755-46d5-8f93-51eb4f71eae2.xml   
354  ecde390e-7b32-4e11-b402-357888850b2e.xml   
355  edcaa815-544c-4743-9b25-85ab7ecdb02c.xml   
356  ee0e318e-8870-4e6e-9886-2fa5d3694a83.xml   
357  ee61e5ed-9def-4d3f-b8e6-824465306959.xml   
358  eec84d33-9bd9-4406-83a6-d04632d71919.xml   
359  eef2c39b-876a-45b5-bcee-b3ee4e912717.xml   
360  efe9686d-2570-41b6-9378-1465171f8f5a.xml   
361  f036dd32-523e-4f35-aa09-b0d7da445591.xml   
362  f16a63fd-688f-4946-a80d-c70fa6055673.xml   
363  f177f16b-4cbe-416b-ba55-7dbe695f0c8a.xml   
364  f1d1ef75-28cb-4c8b-8e50-ace59cb5dd01.xml   
365  f1db4c7b-0833-4546-a506-c208cd906a8c.xml   
366  f1f03bf5-086e-40c5-97ff-8495eee39537.xml   
367  f241994c-36b5-4ee8-b4fd-bc804a6d98ed.xml   
368  f30ebea2-7285-46cd-8047-8f9920f91cb1.xml   
369  f6def6e5-a2c7-419d-afe9-771b5a7b4958.xml   
370  f79924ed-1362-4db7-b235-e7dfd0c333ca.xml   
371  f7e3476c-5f36-4c9c-81ca-9eb4e8b5c20f.xml   
372  f83115e4-24bd-49c6-8806-cb29c6fc627a.xml   
373  f8a2bb4e-9ed9-421f-b9e1-bbfdc48844e0.xml   
374  f926e4a1-7466-496f-a33d-88a1e969be33.xml   
375  f9bfe54d-3ea6-4e0e-951f-dc9175411ceb.xml   
376  faade798-71d0-4d15-b6d8-686226a65a4c.xml   
377  fac0a612-0ab0-4175-8dd4-e0d6944d5c1a.xml   
378  fd299908-fbb1-4ebc-ac50-4c15dbac0529.xml   

                                                 Title  \
0    RORγ antagonists and inverse agonists: a paten...   
1    Secukinumab (AIN457) in the treatment of ankyl...   
2    A safety review of biologic therapies for the ...   
3    Preclinical and clinical characterization of t...   
4    Meta-Analysis of Two Human RNA-seq Datasets to...   
5      Off-label dermatologic uses of IL-17 inhibitors   
6    Interaction of clinical-stage antibodies with ...   
7    Design and baseline characteristics of the Par...   
8    Latest Progresses in Allergic Diseases Biomark...   
9    Advances in biologic and small molecule therap...   
10   Biologics for chronic inflammatory skin diseas...   
11   Small molecules and antibodies for the treatme...   
12   From Bed to Bench and Back: TNF-α, IL-23/IL-17...   
13   Interleukin 17-A inhibition in the treatment o...   
14   Clinical efficacy and IL-17 targeting mechanis...   
15   Much More Than IL-17A: Cytokines of the IL-17 ...   
16   Investigatio

In [142]:
NLM_data[NLM_data['DOI']=='None']

Empty DataFrame
Columns: [XML, Title, Abstract_full, Full_body_text, DOI, URL, Molecule, Publisher_type]
Index: []

In [144]:
APP_FOLDER = 'Downloads/HAWK_AI_2022/Brodalumab/NLM'

totalFiles = 0
totalDir = 0

for base, dirs, files in os.walk(APP_FOLDER):
#     print('Searching in : ',base)
    for directories in dirs:
        totalDir += 1
    for Files in files:
        if Files.endswith('.xml'):
            totalFiles += 1


print('Total number of files',totalFiles)
print('Total Number of directories',totalDir)
print('Total:',(totalDir + totalFiles))

Total number of files 8350
Total Number of directories 0
Total: 8350


In [145]:
writer = pd.ExcelWriter('Downloads/HAWK_AI_2022/Brodalumab/Brodalumab_NLM.xlsx', engine='xlsxwriter',options={'strings_to_urls': False})
NLM_data.to_excel(writer,index=False,sheet_name='Brodalumab')
writer.close()